## Import lib

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly
import plotly.graph_objects as go
import cv2
import os, sys
from zkyhaxpy import io_tools

## Define functions

## Load data

In [ ]:
df_predict_pm25_monthly_median = pd.read_parquet(r'../data/df_predict_pm25_monthly_median.parquet')
df_predict_pm25_monthly_mean = pd.read_parquet(r'../data/df_predict_pm25_monthly_mean.parquet')

df_predict_pm25_monthly_median = df_predict_pm25_monthly_median.median(axis=0).reset_index()
df_predict_pm25_monthly_median.columns = ['year_month', 'pm25_median']

df_predict_pm25_monthly_mean = df_predict_pm25_monthly_mean.mean(axis=0).reset_index()
df_predict_pm25_monthly_mean.columns = ['year_month', 'pm25_mean']

df_predict_pm25_monthly = df_predict_pm25_monthly_mean.merge(df_predict_pm25_monthly_median, left_on='year_month', right_on='year_month')
df_predict_pm25_monthly.iloc[:, 1:] = df_predict_pm25_monthly.iloc[:, 1:].round(2)
df_predict_pm25_monthly.loc[len(df_predict_pm25_monthly)]= pd.Series({'year_month':'2024-05', 'pm25_mean':np.nan, 'pm25_median':np.nan})

df_predict_pm25_monthly.to_csv(r'../data/df_predict_pm25_monthly.csv')

In [ ]:
gdf_openaq = gpd.read_file('../data/gdf_openaq.gpkg')
print('gdf_openaq has been loaded.')
gdf_openaq

In [ ]:
df_tmp = pd.read_parquet(r'C:\Users\surasakc\OneDrive - Bank of Thailand\11-PIER-Blog\Github-Code\2024-04-10 Thailand PM25 - Satellite data\data\chiangmai_aod_daily\df_chiangmai_aod_daily-2024-04-h27v07.parquet')
df_tmp

In [ ]:
list_files_openaq_chiangmai = io_tools.get_list_files('../data/chiangmai_openaq')
list_files_openaq_chiangmai
df_openaq_chiangmai = pd.concat([pd.read_csv(file) for file in list_files_openaq_chiangmai])
df_openaq_chiangmai

In [ ]:
df_openaq_chiangmai.entity.unique()

In [ ]:
df_openaq_chiangmai['year_month'] = df_openaq_chiangmai['date_utc'].str.slice(0, 8)
df_openaq_chiangmai[(df_openaq_chiangmai['entity']=='Governmental Organization')&(df_openaq_chiangmai['date_utc']<='2024-04-09')].groupby(['year_month'], as_index=False).agg(pm25=('value', 'mean')).plot()

In [ ]:
gdf_openaq[gdf_openaq['locationId'] == 225669]

In [ ]:
df_pm25_chiangmai_openaq = pd.read_parquet(r'../data/df_pm25_chiangmai_openaq.parquet')
df_pm25_chiangmai_openaq_monthly_mean = df_pm25_chiangmai_openaq.groupby('year_month').agg(openaq_pm25_mean=('pm25', 'mean'))
df_pm25_chiangmai_openaq_monthly_mean

In [ ]:
# xticks = [xtick for xtick in df_predict_pm25_monthly['year_month'] if xtick.endswith('-01')]
# xticks_text = [xtick[:4] for xtick in xticks]


xticks = ['2005-01', '2010-01', '2015-01', '2020-01', '2024-01']
xticks_text = ['2005', '2010', '2015', '2020', '2024']

In [ ]:
import plotly.express as px

agg = 'mean'
fig = px.line(df_predict_pm25_monthly, x="year_month", y=f"pm25_{agg}", title=f'Monthly PM2.5 ({agg}) in Chiangmai')

fig.update_layout(xaxis_title='Year-Month')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = xticks,
        ticktext = xticks_text
    )
)

fig.update_layout(yaxis_title='PM2.5 (µg/m3)')
fig.update_layout(yaxis_range=(0, 180))
fig.update_traces(line_color='red')

fig.show()
fig.write_json(fr'./time_series-{agg}.json')
fig.write_html(fr'./time_series-{agg}.html', include_plotlyjs=False, full_html=False, div_id=f'time_series-{agg}')